In [5]:
using LinearAlgebra
using Optim
using HDF5

In [6]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    ϵ, h_Re, h_Im = params[1:3]
    γ = params[4:6]
    a = params[7:9]
    
    H = [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2

    C = [
        -γ[1] + γ[2] + γ[3]  -im * a[3]            im * a[2]
        im * a[3]            γ[1] - γ[2] + γ[3]   -im * a[1]
        -im * a[2]           im * a[1]             γ[1] + γ[2] - γ[3]
    ]

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) - 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += norm(ρ1 - ρ2)^2
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [8]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γ = "0.25133"
γᶠ = parse(ComplexF64, γ)

#tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
#tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)

ErrorException: unable to determine if ../DATA/ALL_GAMMAS_B4_D10.h5 is accessible in the HDF5 format (file may not exist)

In [7]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  # Replace with actual initial values




# Define the objective function wrapper
function objective(params)
    return kossak_obj_infid_constrs(params, ρˣ, tˣ)
end

# Perform the optimization
result = optimize(objective, initial_params, BFGS())

# Extract optimized parameters
optimized_params = result.minimizer

println("Optimized parameters: ", optimized_params)

ErrorException: unable to determine if ../DATA/ALL_GAMMAS_B4_D10.h5 is accessible in the HDF5 format (file may not exist)